In [ ]:
# Initialize Workspace
from azureml.core import Workspace
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
# Create or Attach AML Compute
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "rlos-cfe-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

In [ ]:
## Use simple script
# CP script-run
# cb to script-run
import shutil, os
source_directory = 'script-run'
shutil.rmtree(source_directory)
os.makedirs(source_directory, exist_ok=True)
shutil.copytree('cb', os.path.join(source_directory, 'cb'))


In [ ]:
%%writefile $source_directory/estimate.py

# %%run $source_directory/hello.py

# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.
import sys
import os
from mpi4py import MPI

print("*********************************************************")
print("Hello Azure ML!")

mounted_input_path = sys.argv[1]
mounted_output_path = sys.argv[2]

print("Argument 1: %s" % mounted_input_path)
print("Argument 2: %s" % mounted_output_path)
print("Number of nodes {}:".format(MPI.COMM_WORLD.Get_size()))
print("Node index {}:".format(MPI.COMM_WORLD.Get_rank()))

import pandas as pd

if MPI.COMM_WORLD.Get_rank() == 0:
    df = pd.read_csv(os.path.join(mounted_input_path, '01.csv'), parse_dates=['t']).set_index('t')
elif MPI.COMM_WORLD.Get_rank() == 1:
    df = pd.read_csv(os.path.join(mounted_input_path, '02.csv'), parse_dates=['t']).set_index('t')

from cb.estimators import ips_snips


def init_ips_snips(r, p, p_log, n):
    result = ips_snips()
    result.add(r, p_log, p, n * int(p > 0))
    return result

policies = ['random', 'baseline1']
for p in policies:
    df[p] = df.apply(lambda r: init_ips_snips(r['r'], r[f"('b', '{p}')"], r['p'], r['n']), axis = 1)

df = df[policies].resample('5min').sum()

print(df)


In [ ]:
def_blob_store = ws.get_default_datastore()
print(def_blob_store)
import sys
for p in sys.path:
    print(p)
print(sys.path)
def_blob_store.upload_files(files = ["test_data/cb/01.csv", "test_data/cb/02.csv"],
                        target_path = '/dataset/cb/',
                       overwrite = True,
                       show_progress = True)

In [ ]:
from azureml.core import Dataset
from azureml.data import OutputFileDatasetConfig

input_data = Dataset.File.from_files(def_blob_store.path('/dataset/cb/*')).as_named_input('input').as_mount()
output = OutputFileDatasetConfig(destination=(def_blob_store, 'sample/outputdataset'))



In [ ]:
from azureml.core import Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.core.conda_dependencies import CondaDependencies

myenv = Environment("myenv")
myenv.docker.enabled = True
myenv.python.conda_dependencies = CondaDependencies.create(pip_packages=['azureml-sdk>=1.12.0', 'pandas','matplotlib','mpi4py'])
myenv.docker.base_image = 'mcr.microsoft.com/azureml/intelmpi2018.3-cuda9.0-cudnn7-ubuntu16.04'
docker_config = DockerConfiguration(use_docker=True)

In [ ]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import MpiConfiguration

src = ScriptRunConfig(source_directory=source_directory, 
                      script='estimate.py', 
                      # to mount the dataset on the remote compute and pass the mounted path as an argument to the training script
                      arguments =[input_data, output],
                      compute_target=compute_target,
                      environment=myenv,
                      distributed_job_config=MpiConfiguration(node_count=2))

In [ ]:
## Build and Submit Experiment
from azureml.core import Experiment
exp = Experiment(ws, 'ScriptRun_parallel')
run = exp.submit(config=src)

In [ ]:
# View Run Details
run.wait_for_completion(show_output=True)